In [ ]:
import torch
import torch.nn as nn
import timm
from torchvision.models.vision_transformer import vit_b_16, ViT_B_16_Weights

class HybridEfficientNetViT(nn.Module):
    def __init__(self, num_classes=2):
        super(HybridEfficientNetViT, self).__init__()

        # EfficientNet-Lite B0 from TIMM
        self.efficientnet = timm.create_model("efficientnet_lite0", pretrained=True, num_classes=0)  
        self.eff_out_features = self.efficientnet.num_features  

        # Vision Transformer (ViT)
        vit_weights = ViT_B_16_Weights.IMAGENET1K_V1
        self.vit = vit_b_16(weights=vit_weights)
        self.vit.heads = nn.Identity()
        self.vit_out_features = self.vit.hidden_dim

        # Combined classifier
        self.classifier = nn.Sequential(
            nn.Linear(self.eff_out_features + self.vit_out_features, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        eff_features = self.efficientnet(x)  
        vit_features = self.vit(x)          
        combined = torch.cat((eff_features, vit_features), dim=1)
        out = self.classifier(combined)
        return out


d:\cancer\cenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HybridEfficientNetViT(num_classes=2).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

from torch.optim.lr_scheduler import StepLR
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)


Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to C:\Users\lenovo/.cache\torch\hub\checkpoints\vit_b_16-c867db91.pth


100%|██████████| 330M/330M [02:12<00:00, 2.62MB/s] 


In [3]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=10):
    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        scheduler.step()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}")

        # Optional: evaluate on validation set every epoch
        evaluate_model(model, val_loader)

def evaluate_model(model, loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Validation Accuracy: {100 * correct / total:.2f}%")


In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

train_dataset = datasets.ImageFolder("skin_split_keras/train", transform=transform)
val_dataset = datasets.ImageFolder("skin_split_keras/val", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=10)

Epoch 1/10, Loss: 0.3719
Validation Accuracy: 84.92%
Epoch 2/10, Loss: 0.2612
Validation Accuracy: 88.04%
Epoch 3/10, Loss: 0.1490
Validation Accuracy: 87.98%
Epoch 4/10, Loss: 0.0788
Validation Accuracy: 86.76%
Epoch 5/10, Loss: 0.0482
Validation Accuracy: 87.59%
Epoch 6/10, Loss: 0.0279
Validation Accuracy: 87.98%
Epoch 7/10, Loss: 0.0201
